# Feature Overlap Analysis: Conditionals vs Universal Quantifiers

This notebook tests the hypothesis that conditional sentences (if-then) and universal quantifiers (all, every, any) share underlying logical features in language models.

**Key Question**: Do "All birds fly" and "If something is a bird, then it flies" activate similar features?

**Setup**: Runtime → Change runtime type → T4 GPU

## 1. Setup and Installation

In [ ]:
# Check GPU
!nvidia-smi | grep "Tesla" || echo "⚠️ No GPU found - please enable GPU in Runtime settings"

# Install required packages
!pip install -q transformers accelerate einops matplotlib-venn scipy scikit-learn statsmodels

print("✓ Packages installed")

In [ ]:
# Clone repository
!rm -rf SAE_Functional
!git clone https://github.com/ychleee/SAE_Functional.git
%cd SAE_Functional

# Mount Google Drive for results
from google.colab import drive
drive.mount('/content/drive')

RESULTS_DIR = '/content/drive/MyDrive/sae_quantifier_analysis'
!mkdir -p {RESULTS_DIR}
print(f"Results will be saved to: {RESULTS_DIR}")

## 2. Load Quantifier-Conditional Dataset

In [ ]:
import csv
import numpy as np
import pandas as pd
from collections import Counter

# Load dataset
texts = []
types = []
has_quantifier = []

with open('data/processed/quantifier_conditional_dataset.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        texts.append(row['text'])
        types.append(row['type'])
        has_quantifier.append(row['has_quantifier'] == 'True')

# Statistics
type_counts = Counter(types)
print("="*60)
print("DATASET STATISTICS")
print("="*60)
print(f"Total sentences: {len(texts)}")
print("\nBreakdown by type:")
for t, count in sorted(type_counts.items()):
    print(f"  • {t:20s}: {count:4d}")

# Show samples
print("\n" + "="*60)
print("SAMPLE SENTENCES")
print("="*60)

# Show one of each type
shown_types = set()
for i, (text, t) in enumerate(zip(texts, types)):
    if t not in shown_types:
        print(f"\n[{t}]")
        print(f"  {text}")
        shown_types.add(t)
        if len(shown_types) >= min(5, len(type_counts)):
            break

## 3. Extract Activations from Language Model

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load model
model_name = "EleutherAI/pythia-70m"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✓ Model loaded")

In [ ]:
def extract_activations(texts, batch_size=32, max_samples=1200):
    """Extract activations from language model."""
    texts = texts[:max_samples]
    activations = []
    
    print(f"Extracting activations for {len(texts)} sentences...")
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        
        # Tokenize
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)
        
        # Get hidden states
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
            hidden = outputs.hidden_states[-1]  # Last layer
            
            # Mean pooling over tokens
            mask = inputs.attention_mask.unsqueeze(-1)
            pooled = (hidden * mask).sum(1) / mask.sum(1)
            
            activations.append(pooled.cpu())
    
    return torch.cat(activations, dim=0)

# Extract activations
activations = extract_activations(texts, batch_size=32, max_samples=1200)
print(f"\n✓ Activations shape: {activations.shape}")

# Convert types to numpy array for easier indexing
types_array = np.array(types[:len(activations)])

## 4. Train Sparse Autoencoder

In [ ]:
import torch.nn as nn
import torch.optim as optim

class SparseAutoencoder(nn.Module):
    """Sparse Autoencoder for feature discovery."""
    
    def __init__(self, input_dim, hidden_dim=2048, sparsity_coeff=0.01):
        super().__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim, bias=False)
        self.sparsity_coeff = sparsity_coeff
        
        # Tie weights
        self.decoder.weight = nn.Parameter(self.encoder.weight.t())
        
        # Initialize
        nn.init.xavier_uniform_(self.encoder.weight)
        nn.init.zeros_(self.encoder.bias)
    
    def encode(self, x):
        return torch.relu(self.encoder(x))
    
    def forward(self, x):
        code = self.encode(x)
        recon = self.decoder(code)
        return recon, code

# Create SAE
input_dim = activations.shape[1]
hidden_dim = 2048  # Larger for more fine-grained features
sae = SparseAutoencoder(input_dim, hidden_dim, sparsity_coeff=0.01).to(device)

print(f"SAE Architecture: {input_dim} → {hidden_dim} → {input_dim}")
print(f"Total parameters: {sum(p.numel() for p in sae.parameters()):,}")

In [ ]:
# Training setup
optimizer = optim.Adam(sae.parameters(), lr=0.001)
acts_tensor = activations.to(device)

# Training loop
print("Training SAE...")
print("="*60)

losses = []
for epoch in range(100):
    # Forward pass
    recon, code = sae(acts_tensor)
    
    # Losses
    recon_loss = nn.functional.mse_loss(recon, acts_tensor)
    sparse_loss = sae.sparsity_coeff * code.abs().mean()
    total_loss = recon_loss + sparse_loss
    
    # Backward pass
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    losses.append(total_loss.item())
    
    if epoch % 20 == 0:
        active = (code > 0).float().mean().item()
        print(f"Epoch {epoch:3d}: Loss={total_loss:.4f} (R:{recon_loss:.4f} S:{sparse_loss:.4f}) Active={active:.3f}")

print("="*60)
print("✓ Training complete!")

## 5. Analyze Feature Overlap

In [ ]:
# Import analysis module
import sys
sys.path.append('..')
from analyze_quantifier_overlap import FeatureOverlapAnalyzer

# Create analyzer
analyzer = FeatureOverlapAnalyzer(sae, device)

# Encode all sentences
print("Encoding sentences to sparse features...")
codes_by_type, all_codes = analyzer.encode_sentences(activations.numpy(), types_array)

print(f"\n✓ Encoded {len(all_codes)} sentences into {all_codes.shape[1]} features")
print(f"  • Conditionals: {len(codes_by_type['conditional'])} sentences")
print(f"  • Quantifiers: {len(codes_by_type['quantifier'])} sentences")
print(f"  • Controls: {len(codes_by_type['control'])} sentences")

In [ ]:
# Identify characteristic features
print("Identifying characteristic features...")
features = analyzer.identify_characteristic_features(codes_by_type, threshold=0.05)

print("\n" + "="*60)
print("FEATURE DISCOVERY RESULTS")
print("="*60)
print(f"\n📊 Feature Categories:")
print(f"  • Conditional-specific features: {len(features['conditional_specific'])}")
print(f"  • Quantifier-specific features: {len(features['quantifier_specific'])}")
print(f"  • SHARED LOGICAL FEATURES: {len(features['shared'])} ⭐")
print(f"  • Control features: {len(features['control'])}")

# Calculate overlap percentage
total_relevant = (len(features['conditional_specific']) + 
                 len(features['quantifier_specific']) + 
                 len(features['shared']))
if total_relevant > 0:
    overlap_pct = len(features['shared']) / total_relevant * 100
    print(f"\n🎯 Overlap: {overlap_pct:.1f}% of relevant features are shared")
    print(f"   Jaccard Index: {features['overlap_stats']['jaccard_index']:.3f}")

# Show top shared features
if len(features['shared']) > 0:
    print("\n🔍 Top Shared Features (encode logical relationships):")
    for i, feat_idx in enumerate(features['shared'][:10]):
        cond_act = features['overlap_stats']['conditional_mean'][feat_idx]
        quant_act = features['overlap_stats']['quantifier_mean'][feat_idx]
        ctrl_act = features['overlap_stats']['control_mean'][feat_idx]
        print(f"   {i+1}. Feature {feat_idx}: Cond={cond_act:.3f}, Quant={quant_act:.3f}, Ctrl={ctrl_act:.3f}")

## 6. Statistical Significance Testing

In [ ]:
# Perform statistical tests
print("Running statistical tests...")
stats_results = analyzer.statistical_tests(codes_by_type)

print("\n" + "="*60)
print("STATISTICAL ANALYSIS")
print("="*60)
print(f"\n📈 Hypothesis Testing:")
print(f"  • H0: Conditionals and quantifiers have identical feature distributions")
print(f"  • H1: They have different but overlapping distributions")

print(f"\n📊 Results:")
print(f"  • Features with significant differences: {len(stats_results['significant_features'])}")
print(f"  • KS test statistic: {stats_results['ks_test']['statistic']:.4f}")
print(f"  • KS test p-value: {stats_results['ks_test']['p_value']:.2e}")

# Effect sizes
large_effects = np.where(np.abs(stats_results['effect_sizes']) > 0.8)[0]
medium_effects = np.where((np.abs(stats_results['effect_sizes']) > 0.5) & 
                         (np.abs(stats_results['effect_sizes']) <= 0.8))[0]

print(f"\n📏 Effect Sizes (Cohen's d):")
print(f"  • Large effects (|d| > 0.8): {len(large_effects)} features")
print(f"  • Medium effects (0.5 < |d| ≤ 0.8): {len(medium_effects)} features")

if stats_results['ks_test']['p_value'] < 0.001:
    print("\n✅ Conclusion: Statistically significant difference in distributions")
    print("   BUT substantial feature overlap suggests shared logical processing")
else:
    print("\n✅ Conclusion: Similar feature distributions")
    print("   Strong evidence for shared logical representations")

## 7. Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Create Venn diagram
print("Creating Venn diagram...")
venn_fig = analyzer.create_venn_diagram(features)
plt.show()

# Save figure
venn_fig.savefig(f'{RESULTS_DIR}/venn_diagram.png', dpi=300, bbox_inches='tight')
print("✓ Venn diagram saved")

In [ ]:
# Create t-SNE visualization
print("Creating t-SNE visualization (this may take a minute)...")
tsne_fig = analyzer.create_tsne_visualization(all_codes, types_array)
plt.show()

# Save figure
tsne_fig.savefig(f'{RESULTS_DIR}/tsne_plot.png', dpi=300, bbox_inches='tight')
print("✓ t-SNE plot saved")

In [ ]:
# Create heatmaps
print("Creating feature heatmaps...")
heatmap_fig = analyzer.create_heatmap(features)
plt.show()

# Save figure
heatmap_fig.savefig(f'{RESULTS_DIR}/heatmaps.png', dpi=300, bbox_inches='tight')
print("✓ Heatmaps saved")

In [ ]:
# Similarity matrix
print("Computing similarity matrix...")
similarity, patterns = analyzer.compute_similarity_matrix(codes_by_type)

# Plot similarity matrix
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(similarity, 
            xticklabels=['Conditional', 'Quantifier', 'Control'],
            yticklabels=['Conditional', 'Quantifier', 'Control'],
            annot=True, fmt='.3f', cmap='YlOrRd', 
            vmin=0, vmax=1, square=True, cbar_kws={'label': 'Cosine Similarity'})
ax.set_title('Feature Pattern Similarity Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Save
fig.savefig(f'{RESULTS_DIR}/similarity_matrix.png', dpi=300, bbox_inches='tight')
print("✓ Similarity matrix saved")

print(f"\n🔗 Conditional-Quantifier Similarity: {similarity[0, 1]:.3f}")
print(f"   Conditional-Control Similarity: {similarity[0, 2]:.3f}")
print(f"   Quantifier-Control Similarity: {similarity[1, 2]:.3f}")

## 8. Feature Examples

In [ ]:
# Find example sentences for top shared features
print("="*60)
print("EXAMPLE SENTENCES FOR SHARED FEATURES")
print("="*60)

if len(features['shared']) > 0:
    # Take top 3 shared features
    for feat_idx in features['shared'][:3]:
        print(f"\n📍 Feature {feat_idx} (Shared Logical Feature)")
        print("-"*40)
        
        # Get activations for this feature
        feat_acts = all_codes[:, feat_idx]
        
        # Find top examples from conditionals
        cond_mask = types_array == 'conditional'
        cond_acts = feat_acts[cond_mask]
        cond_texts = [t for t, m in zip(texts[:len(types_array)], cond_mask) if m]
        
        if len(cond_acts) > 0:
            top_cond = np.argsort(cond_acts)[-2:][::-1]
            print("\n  Conditionals:")
            for idx in top_cond:
                if idx < len(cond_texts):
                    print(f"    [{cond_acts[idx]:.2f}] {cond_texts[idx][:80]}")
        
        # Find top examples from quantifiers
        quant_mask = np.isin(types_array, ['pure_universal', 'restricted_universal', 
                                           'negative_universal', 'generic_universal', 
                                           'any_universal'])
        quant_acts = feat_acts[quant_mask]
        quant_texts = [t for t, m in zip(texts[:len(types_array)], quant_mask) if m]
        quant_types = [t for t, m in zip(types_array, quant_mask) if m]
        
        if len(quant_acts) > 0:
            top_quant = np.argsort(quant_acts)[-2:][::-1]
            print("\n  Universal Quantifiers:")
            for idx in top_quant:
                if idx < len(quant_texts):
                    q_type = quant_types[idx].replace('_', ' ')
                    print(f"    [{quant_acts[idx]:.2f}] [{q_type}] {quant_texts[idx][:60]}")
else:
    print("No shared features found with current threshold.")
    print("This might indicate distinct processing mechanisms.")

## 9. Generate Final Report

In [ ]:
# Generate comprehensive report
report = analyzer.generate_report(codes_by_type, features, stats_results)
print(report)

# Save report
with open(f'{RESULTS_DIR}/analysis_report.txt', 'w') as f:
    f.write(report)
print(f"\n✓ Report saved to {RESULTS_DIR}/analysis_report.txt")

## 10. Save Model and Results

In [ ]:
import json
from datetime import datetime

# Save model
torch.save({
    'model_state': sae.state_dict(),
    'input_dim': input_dim,
    'hidden_dim': hidden_dim,
    'sparsity_coeff': sae.sparsity_coeff
}, f'{RESULTS_DIR}/sae_model.pt')
print(f"✓ Model saved")

# Save analysis results
results = {
    'timestamp': datetime.now().isoformat(),
    'dataset': {
        'total_sentences': len(texts),
        'conditionals': int(type_counts.get('conditional', 0)),
        'quantifiers': sum(v for k, v in type_counts.items() if 'universal' in k),
        'controls': int(type_counts.get('control', 0))
    },
    'features': {
        'conditional_specific': len(features['conditional_specific']),
        'quantifier_specific': len(features['quantifier_specific']),
        'shared': len(features['shared']),
        'jaccard_index': float(features['overlap_stats']['jaccard_index'])
    },
    'statistical_tests': {
        'ks_statistic': float(stats_results['ks_test']['statistic']),
        'ks_p_value': float(stats_results['ks_test']['p_value']),
        'significant_features': int(len(stats_results['significant_features']))
    },
    'similarity': {
        'conditional_quantifier': float(similarity[0, 1]),
        'conditional_control': float(similarity[0, 2]),
        'quantifier_control': float(similarity[1, 2])
    }
}

with open(f'{RESULTS_DIR}/results.json', 'w') as f:
    json.dump(results, f, indent=2)
print(f"✓ Results saved")

print("\n" + "="*60)
print("🎉 ANALYSIS COMPLETE!")
print("="*60)
print(f"\nAll results saved to: {RESULTS_DIR}")
print("\nFiles created:")
print("  • venn_diagram.png - Feature overlap visualization")
print("  • tsne_plot.png - Sentence clustering in feature space")
print("  • heatmaps.png - Feature activation patterns")
print("  • similarity_matrix.png - Type similarity analysis")
print("  • analysis_report.txt - Comprehensive text report")
print("  • sae_model.pt - Trained SAE model")
print("  • results.json - Numerical results")
print("\n📥 Download from Google Drive when complete!")